<a href="https://colab.research.google.com/github/girirajgo/Chatbot/blob/master/Chatbot%20using%20nltk%20and%20stemmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json

with open("intents.json") as file:
    data=json.load(file)

print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?'], 'responses': ["We're open every day 9am-9pm", 'Our hours are 9am-9pm every day']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and AMEX', 'We accept most major credit cards']}, {'tag': 'opentoday', 'patterns': ['Are you open today?', 'When do you op

In [ ]:
pip install tensorflow==1.13.2



SyntaxError: ignored

In [ ]:
pip install tflearn


In [ ]:
pip install tensorflow==1.13.2


     |████████████████████████████████| 92.6MB 61kB/s 
     |████████████████████████████████| 3.2MB 46.3MB/s 
     |████████████████████████████████| 368kB 46.6MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow as tf
import random
import json
with open("intents.json") as file:
    data=json.load(file)
words=[]
labels=[]
docs_x=[]
docs_y=[]

for intent in data['intents']:
  for pattern in intent['patterns']:
    wrds=nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
      labels.append(intent['tag'])
words=[stemmer.stem(w.lower()) for w in words if w not in "?"]
words=sorted(list(set(words)))
labels=sorted(labels)

training=[]
output=[]
out_empty =[0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag=[]
  wrds=[stemmer.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)
  output_row=out_empty[:]
  output_row[labels.index(docs_y[x])]=1
  training.append(bag)
  output.append(output_row)

training= numpy.array(training)
output=numpy.array(output)

tf.reset_default_graph()
net=tflearn.input_data(shape=[None, len(training[0])])
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,8)
net=tflearn.fully_connected(net,len(output[0]), activation="softmax")
net=tflearn.regression(net)
model=tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()


Training Step: 2999  | total loss: 0.26205 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.26205 - acc: 0.9725 -- iter: 16/20
Training Step: 3000  | total loss: 0.24820 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.24820 - acc: 0.9777 -- iter: 20/20
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
Start talking with the bot (type quit to stop)!
You: quit
